In [1]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import re

filter ppp products. keep atleast 2-3 product generations back

In [2]:
ppp_df = pd.read_csv('data/raw/ppp/cpu.csv')

In [ ]:
def is_recent_cpu(name):
    name_lower = name.lower()
    if re.search(r"i[3579]-(10|11|1[2-5])\d{3}", name_lower):
        return True
    if re.search(r"ryzen [3579] (5|7|8|9)\d{3}", name_lower):
        return True
    if re.search(r"core ultra", name_lower):
        return True
    return False


In [4]:
ppp_df = ppp_df[ppp_df['name'].apply(is_recent_cpu)]
ppp_df = ppp_df.reset_index(drop = True)

In [5]:
ppp_df.to_csv('data/filtered/ppp_filtered_cpu.csv')

extract local dataset

In [6]:
local_cpus = pd.read_csv('data/filtered/prices_cpu.csv', names=['item_name','category_name','price_php','stocks'])

local_cpus.head()


,item_name,category_name,price_php,stocks
0,item_name,category_name,item_price_in_php,stocks
1,"Intel Core i3 10105F 4C/8T 6mb 65W LGA1200, Box",Processor,4999.0,5
2,Intel Core i5 10400 6C/12T 12mb UHD630 65W LGA...,Processor,7699.0,0
3,"Intel Core i3 13100F 4C/8T 12mb 58W LGA1700, Box",Processor,5799.0,16
4,Intel Core i3 14100 4C/8T 12mb UHD730 60W LGA1...,Processor,8699.0,8


In [ ]:
def extract_cpu_model(name):
    name = name.lower()

    intel_match = re.search(r"intel core i[3579] \d{4,5}[a-z]{0,3}", name)
    if intel_match:
        return intel_match.group(0).title()

    ultra_match = re.search(r"intel core ultra [579] \d+[a-z]{0,3}", name)
    if ultra_match:
        return ultra_match.group(0).title()

    amd_match = re.search(r"amd ryzen [3579] \d{4,5}[a-z]?", name)
    if amd_match:
        return amd_match.group(0).title()

    return None 

local_cpus["model_key"] = local_cpus["item_name"].apply(extract_cpu_model)

local_cpus = local_cpus.dropna(subset=["model_key"])

local_cpus.to_csv('data/filtered/prices_cpu.csv')


In [16]:
def normalize_ppp_name(name):
    name_clean = re.sub(r"-", " ", name)

    name_clean = re.sub(r"\s+", " ", name_clean) 
    return name_clean.strip().title()

ppp_df['model_key'] = ppp_df['name'].apply(normalize_ppp_name)

ppp_df.head()

,name,price,core_count,core_clock,boost_clock,microarchitecture,tdp,graphics,model_key
0,AMD Ryzen 7 9800X3D,451.50,8,4.7,5.2,Zen 5,120,Radeon,Amd Ryzen 7 9800X3D
1,AMD Ryzen 7 7800X3D,340.05,8,4.2,5.0,Zen 4,120,Radeon,Amd Ryzen 7 7800X3D
2,AMD Ryzen 5 7600X,170.49,6,4.7,5.3,Zen 4,105,Radeon,Amd Ryzen 5 7600X
3,AMD Ryzen 5 9600X,204.99,6,3.9,5.4,Zen 5,65,Radeon,Amd Ryzen 5 9600X
4,AMD Ryzen 7 7700X,242.98,8,4.5,5.4,Zen 4,105,Radeon,Amd Ryzen 7 7700X


In [26]:
merged_cpus = pd.merge(ppp_df, local_cpus[['model_key', 'price_php']], 
                       on="model_key", how="inner")

merged_cpus.to_csv("data/processed/cpus_merged.csv", index=False)

In [28]:
def assign_socket(cpu_name: str) -> str:
    name = cpu_name.lower()

    if re.search(r"i[3579]-10\d{3}|i[3579]-11\d{3}", name):
        return "LGA1200"

    if re.search(r"i[3579]-1[2-5]\d{3}", name):
        return "LGA1700"

    if re.search(r"core ultra \d{1,3}", name):
        return "LGA1851"

    if re.search(r"ryzen [3579] 5\d{3}", name):
        return "AM4"

    if re.search(r"ryzen [3579] 7\d{3}", name):
        return "AM5"
    
    if re.search(r"ryzen [3579] 8\d{3}", name):
        return "AM5"
    
    if re.search(r"ryzen [3579] 9\d{3}", name):
        return "AM5"

    return "Unknown"

# Apply to DataFrame
merged_cpus['socket'] = merged_cpus['name'].apply(assign_socket)
merged_cpus = merged_cpus.drop(columns=['price', 'model_key'])

merged_cpus.to_csv("data/processed/cpus_merged.csv", index=False)